**<h1> <font color="#FF416C">IMPORT FRAMEWORK AND LIBRARY**

In [ ]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import re
import math
import string
import random
import numpy as np
import random as rd
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType, DoubleType
from pyspark.sql.functions import least
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans

**<h1> <font color="#FF416C">CONST VALUE**

In [ ]:
_THRESHOLD = 0.01

# number of cluster K
K = 3

**<h1> <font color="#FF416C">IMPORT DATA**

In [ ]:
spark = SparkSession.builder.appName('Credit Card Clustering').getOrCreate()

23/12/20 22:45:50 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/12/20 22:45:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/20 22:45:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/20 22:45:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/20 22:45:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
path_data = '/cc2.csv'

In [ ]:
_schema = StructType([\
                     StructField("CUST_ID", StringType(), True),
                     StructField("BALANCE", DoubleType(), True),
                     StructField("BALANCE_FREQUENCY", DoubleType(), True),
                     StructField("PURCHASES", DoubleType(), True),
                     StructField("ONEOFF_PURCHASES", DoubleType(), True),
                     StructField("INSTALLMENTS_PURCHASES", DoubleType(), True),
                     StructField("CASH_ADVANCE", DoubleType(), True),
                     StructField("PURCHASES_FREQUENCY", DoubleType(), True),
                     StructField("ONEOFF_PURCHASES_FREQUENCY", DoubleType(), True),
                     StructField("PURCHASES_INSTALLMENTS_FREQUENCY", DoubleType(), True),
                     StructField("CASH_ADVANCE_FREQUENCY", DoubleType(), True),
                     StructField("CASH_ADVANCE_TRX", IntegerType(), True),
                     StructField("PURCHASES_TRX", IntegerType(), True),
                     StructField("CREDIT_LIMIT", DoubleType(), True),
                     StructField("PAYMENTS", DoubleType(), True),
                     StructField("MINIMUM_PAYMENTS", DoubleType(), True),
                     StructField("PRC_FULL_PAYMENT", DoubleType(), True),
                     StructField("TENURE", IntegerType(), True)])

In [ ]:
_data = spark\
      .read.option("delimiter", ",")\
      .option("header", True)\
      .schema(_schema)\
      .csv(path_data)

In [ ]:
_data.show()

+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|CUST_ID|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
| C10001|  40.900749|         0.818182|     95.4|             0.0|                  95.4|  

**<h1> <font color="#FF416C">DATA PREPROCESSING**

In [ ]:
_data.explain()

== Physical Plan ==
FileScan csv [CUST_ID#0,BALANCE#1,BALANCE_FREQUENCY#2,PURCHASES#3,ONEOFF_PURCHASES#4,INSTALLMENTS_PURCHASES#5,CASH_ADVANCE#6,PURCHASES_FREQUENCY#7,ONEOFF_PURCHASES_FREQUENCY#8,PURCHASES_INSTALLMENTS_FREQUENCY#9,CASH_ADVANCE_FREQUENCY#10,CASH_ADVANCE_TRX#11,PURCHASES_TRX#12,CREDIT_LIMIT#13,PAYMENTS#14,MINIMUM_PAYMENTS#15,PRC_FULL_PAYMENT#16,TENURE#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/cc2.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<CUST_ID:string,BALANCE:double,BALANCE_FREQUENCY:double,PURCHASES:double,ONEOFF_PURCHASES:d...




In [ ]:
_data.printSchema()

root
 |-- CUST_ID: string (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- BALANCE_FREQUENCY: double (nullable = true)
 |-- PURCHASES: double (nullable = true)
 |-- ONEOFF_PURCHASES: double (nullable = true)
 |-- INSTALLMENTS_PURCHASES: double (nullable = true)
 |-- CASH_ADVANCE: double (nullable = true)
 |-- PURCHASES_FREQUENCY: double (nullable = true)
 |-- ONEOFF_PURCHASES_FREQUENCY: double (nullable = true)
 |-- PURCHASES_INSTALLMENTS_FREQUENCY: double (nullable = true)
 |-- CASH_ADVANCE_FREQUENCY: double (nullable = true)
 |-- CASH_ADVANCE_TRX: integer (nullable = true)
 |-- PURCHASES_TRX: integer (nullable = true)
 |-- CREDIT_LIMIT: double (nullable = true)
 |-- PAYMENTS: double (nullable = true)
 |-- MINIMUM_PAYMENTS: double (nullable = true)
 |-- PRC_FULL_PAYMENT: double (nullable = true)
 |-- TENURE: integer (nullable = true)



In [ ]:
print((_data.count(), len(_data.columns)))

(8950, 18)


In [ ]:
_data = _data.select([col for col in _data.columns if col != 'CUST_ID'])

**<h3> <font color="#8f94fb">Calculating basic indexes for a Dataframe**


In [ ]:
_data.summary().show()

23/12/20 22:45:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------------------+------------------+------------------+----------------------+------------------+-------------------+--------------------------+--------------------------------+----------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+
|summary|           BALANCE|  BALANCE_FREQUENCY|         PURCHASES|  ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|      CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|  CASH_ADVANCE_TRX|     PURCHASES_TRX|     CREDIT_LIMIT|          PAYMENTS| MINIMUM_PAYMENTS|  PRC_FULL_PAYMENT|           TENURE|
+-------+------------------+-------------------+------------------+------------------+----------------------+------------------+-------------------+--------------------------+--------------------------------+----------------------+------------------+------------------+-------

In [ ]:
def median_formula(data_frame: DataFrame, column_name):
        # Calculate median value of the column name
        median_result = data_frame.agg(percentile_approx(column_name, 0.5, lit(1000000)).alias("Median"))

        # Return a dataframe
        return median_result

In [ ]:
median_formula(_data, "BALANCE").show()

+----------+
|    Median|
+----------+
|873.090183|
+----------+



In [ ]:
def mode_formula(data_frame: DataFrame, column_name):
        # Group by the specified column and count occurrences
        mode_result = data_frame.groupBy(column_name).agg(count(column_name).alias('count'))
        # Calculate mode value of the column name
        mode_result = mode_result.withColumn('rank', row_number().over(Window.orderBy(desc('count'))))
        mode_result = mode_result.filter(mode_result['rank'] == 1).drop('rank', 'count')

        # Return a dataframe
        return mode_result

In [ ]:
mode_formula(_data, "CREDIT_LIMIT").show()

23/12/20 22:46:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/20 22:46:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/20 22:46:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/20 22:46:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/20 22:46:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/20 22:46:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+
|CREDIT_LIMIT|
+------------+
|      3000.0|
+------------+



In [ ]:
def variance_formula(data_frame: DataFrame, column_name):
        # Calculate median value of the column name
        variance_result = data_frame.select(variance(column_name))

        # Return a dataframe
        return variance_result

In [ ]:
variance_formula(_data, "BALANCE").show()

+-----------------+
|var_samp(BALANCE)|
+-----------------+
|4332774.965193922|
+-----------------+



In [ ]:
def asc_formula(data_frame: DataFrame, column_name):
        # Calculate median value of the column name
        asc_result = data_frame.sort(asc(column_name))

        # Return a dataframe
        return asc_result

In [ ]:
asc_formula(_data, "BALANCE").show()

+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|    0.0|              0.0|   199.73|            22.0|                177.73|         0.0|           0.454545|                 

In [ ]:
def desc_formula(data_frame: DataFrame, column_name):
        # Calculate median value of the column name
        desc_result = data_frame.orderBy(desc(column_name))

        # Return a dataframe
        return desc_result

In [ ]:
desc_formula(_data, "BALANCE").show()

+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|19043.13856|              1.0| 22009.92|         9449.07|              12560.85|         0.0|                1.0| 

In [ ]:
def last_value_formula(data_frame: DataFrame, column_name):
        # Calculate median value of the column name
        last_value_result = data_frame.select(last(column_name))

        # Return a dataframe
        return last_value_result

In [ ]:
last_value_formula(_data, "BALANCE").show()

+-------------+
|last(BALANCE)|
+-------------+
|   372.708075|
+-------------+



In [ ]:
def last_row_formula(data_frame: DataFrame):
        # Calculate median value of the column name
        last_row_result = data_frame.withColumn('id', monotonically_increasing_id())\
             .select(max(struct('id', *data_frame.columns))\
             .alias('x')).select(col('x.*')).drop('id')

        # Return a dataframe
        return last_row_result

In [ ]:
last_row_formula(_data).show()

+----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+---------+----------------+----------------+------+
|   BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT| PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+---------+----------------+----------------+------+
|372.708075|         0.666667|  1093.25|         1093.25|                   0.0|  127.040008|           0.666667|           

In [ ]:
def quartiles_formula(data_frame:DataFrame, column_name):
        quartiles_result = data_frame.approxQuantile(column_name, [0.25, 0.5, 0.75], 0)

        # Return a list
        return quartiles_result

def outliers_formula(data_frame:DataFrame, column_name, factor  = 1.5):
        q1, q3 = quartiles_formula(data_frame, column_name)[0], quartiles_formula(data_frame, column_name)[2]
        iqr = q3 - q1
        lower_bound = (q1 - factor * iqr)
        upper_bound = q3 + factor * iqr
        outliner_result = data_frame.filter((col(column_name) < lower_bound) | (col(column_name) > upper_bound))
        print(f"DataFrame with lower bound ({lower_bound}) <= {column_name} <= upper bound ({upper_bound}):")

        # Return a dataframe contain outliner
        return outliner_result

In [ ]:
outliers_formula(_data, "BALANCE").show()

DataFrame with lower bound (-2760.924372) <= BALANCE <= upper bound (4943.55118):
+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|6886.213231|              1.0|  

In [ ]:
def trending_formula(data_frame: DataFrame, column_name_1, column_name_2):
        window_spec = Window.partitionBy(column_name_1)
        sum_total_amount = sum(column_name_2).over(window_spec).alias('Total')

        trending_result = data_frame.select(column_name_1, sum_total_amount).distinct()
        trending_result = trending_result.orderBy(col('Total').desc())
        return trending_result

In [ ]:
trending_formula(_data, 'TENURE', 'PAYMENTS' ).show()

+------+--------------------+
|TENURE|               Total|
+------+--------------------+
|    12|1.4057753027644007E7|
|    11|   572573.9025190002|
|    10|   266024.0543859998|
|     9|  169357.53850599995|
|     8|  165186.32896100005|
|     7|  150817.82705000002|
|     6|  129924.79655599994|
+------+--------------------+



**<h3> <font color="#8f94fb">Dealing with Null Values**

In [ ]:
_data.select([F.count( F.when( F.isnan(column) | F.col(column).isNull(), column)).alias(column) for column in _data.columns]).show()

+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+--------+----------------+----------------+------+
|BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+--------+----------------+----------------+------+
|      0|                0|        0|               0|                     0|           0|                  0|                         0

In [ ]:
clear_data_1 = _data.where(_data['CREDIT_LIMIT'].isNotNull())

In [ ]:
clear_data_1.select([F.count( F.when( F.isnan(column) | F.col(column).isNull(), column)).alias(column) for column in clear_data_1.columns]).show(truncate=False)

+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+--------+----------------+----------------+------+
|BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+--------+----------------+----------------+------+
|0      |0                |0        |0               |0                     |0           |0                  |0                         

In [ ]:
clear_data_1.groupBy("MINIMUM_PAYMENTS").agg(F.count("MINIMUM_PAYMENTS").alias("num_row")).sort(F.col("num_row").desc()).show(clear_data_1.count(),truncate=False)

+----------------+-------+
|MINIMUM_PAYMENTS|num_row|
+----------------+-------+
|299.351881      |2      |
|128.834658      |1      |
|930.115009      |1      |
|885.376298      |1      |
|1287.324868     |1      |
|152.391012      |1      |
|285.440312      |1      |
|1860.347671     |1      |
|2716.326216     |1      |
|179.244755      |1      |
|1809.974326     |1      |
|886.458108      |1      |
|179.358734      |1      |
|526.749661      |1      |
|218.463565      |1      |
|233.202069      |1      |
|77.604225       |1      |
|292.103091      |1      |
|434.689529      |1      |
|237.529082      |1      |
|135.625973      |1      |
|336.850732      |1      |
|131.349565      |1      |
|119.964053      |1      |
|161.497921      |1      |
|1555.793076     |1      |
|191.258003      |1      |
|761.902854      |1      |
|1600.26917      |1      |
|350.085005      |1      |
|176.822347      |1      |
|4192.565071     |1      |
|87.824996       |1      |
|200.597917      |1      |
|

In [ ]:
fill_data_1 = clear_data_1.fillna(864, "MINIMUM_PAYMENTS")

In [ ]:
fill_data_1.select([F.count( F.when( F.isnan(column) | F.col(column).isNull(), column)).alias(column) for column in fill_data_1.columns]).show(truncate=False)

+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+--------+----------------+----------------+------+
|BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+--------+----------------+----------------+------+
|0      |0                |0        |0               |0                     |0           |0                  |0                         

**<h3> <font color="#8f94fb">Transform Data**

In [ ]:
def standardize_dataframe(dataframe):
    numeric_columns = [
        column[0] for column in dataframe.dtypes
        if column[1] in ('int', 'bigint', 'float', 'double')
    ]

    means_stddevs = {}
    for column in numeric_columns:
        mean_value = dataframe.select(mean(col(column))).collect()[0][0]
        stddev_value = dataframe.select(stddev(col(column))).collect()[0][0]
        means_stddevs[column] = (mean_value, stddev_value)

    standardized_data = dataframe
    for column, (mean_value, stddev_value) in means_stddevs.items():
        new_column_name = f"{column}_trans"
        standardized_data = standardized_data.withColumn(
            new_column_name, round((col(column) - mean_value) / stddev_value,4)
        ).drop(column)

    return standardized_data

In [ ]:
def normalize_dataframe(dataframe):
    numeric_columns = [
        column[0] for column in dataframe.dtypes
        if column[1] in ('int', 'bigint', 'float', 'double')
    ]

    min_max_values = {}
    for column in numeric_columns:
        min_value = dataframe.select(min(col(column))).collect()[0][0]
        max_value = dataframe.select(max(col(column))).collect()[0][0]
        min_max_values[column] = (min_value, max_value)

    normalized_data = dataframe
    for column, (min_value, max_value) in min_max_values.items():
        new_column_name = f"{column}_normalized"
        normalized_data = normalized_data.withColumn(
            new_column_name, (col(column) - min_value) / (max_value - min_value)
        ).drop(column)

    return normalized_data

In [ ]:
df_transform = standardize_dataframe(fill_data_1)
df_transform.show()

+-------------+-----------------------+---------------+----------------------+----------------------------+------------------+-------------------------+--------------------------------+--------------------------------------+----------------------------+----------------------+-------------------+------------------+--------------+----------------------+----------------------+------------+
|BALANCE_trans|BALANCE_FREQUENCY_trans|PURCHASES_trans|ONEOFF_PURCHASES_trans|INSTALLMENTS_PURCHASES_trans|CASH_ADVANCE_trans|PURCHASES_FREQUENCY_trans|ONEOFF_PURCHASES_FREQUENCY_trans|PURCHASES_INSTALLMENTS_FREQUENCY_trans|CASH_ADVANCE_FREQUENCY_trans|CASH_ADVANCE_TRX_trans|PURCHASES_TRX_trans|CREDIT_LIMIT_trans|PAYMENTS_trans|MINIMUM_PAYMENTS_trans|PRC_FULL_PAYMENT_trans|TENURE_trans|
+-------------+-----------------------+---------------+----------------------+----------------------------+------------------+-------------------------+--------------------------------+-----------------------------------

In [ ]:
train = df_transform.withColumn("label", (col("BALANCE_FREQUENCY_trans") >= 0.5).cast("string"))
train = train.withColumn("target", when(col("label") == "false", 1).otherwise(0))

train.show(10)

+-------------+-----------------------+---------------+----------------------+----------------------------+------------------+-------------------------+--------------------------------+--------------------------------------+----------------------------+----------------------+-------------------+------------------+--------------+----------------------+----------------------+------------+-----+------+
|BALANCE_trans|BALANCE_FREQUENCY_trans|PURCHASES_trans|ONEOFF_PURCHASES_trans|INSTALLMENTS_PURCHASES_trans|CASH_ADVANCE_trans|PURCHASES_FREQUENCY_trans|ONEOFF_PURCHASES_FREQUENCY_trans|PURCHASES_INSTALLMENTS_FREQUENCY_trans|CASH_ADVANCE_FREQUENCY_trans|CASH_ADVANCE_TRX_trans|PURCHASES_TRX_trans|CREDIT_LIMIT_trans|PAYMENTS_trans|MINIMUM_PAYMENTS_trans|PRC_FULL_PAYMENT_trans|TENURE_trans|label|target|
+-------------+-----------------------+---------------+----------------------+----------------------------+------------------+-------------------------+--------------------------------+---------

**<h3> <font color="#8f94fb">Classification Model**

In [ ]:
columns_to_exclude = ['target', 'CUST_ID', 'label']
selected_columns = [col for col in train.columns if col not in columns_to_exclude]


# Add an index column
train = train.withColumn("index", monotonically_increasing_id())

train_class = train.withColumn("features", array(*selected_columns))


train_class.select("index", "features", "target").show(5, truncate=False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------+------+
|index|features                                                                                                                                             |target|
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------+------+
|0    |[-0.732, -0.2499, -0.4249, -0.3569, -0.3491, -0.4668, -0.8066, -0.6787, -0.7074, -0.6753, -0.4761, -0.5114, -0.9603, -0.529, -0.311, -0.5256, 0.3605]|1     |
|1    |[0.7868, 0.134, -0.4696, -0.3569, -0.4546, 2.6053, -1.2219, -0.6787, -0.917, 0.5739, 0.11, -0.5918, 0.6886, 0.8185, 0.0893, 0.2341, 0.3605]          |1     |
|2    |[0.447, 0.518, -0.1077, 0.1088, -0.4546, -0.4668, 1.2697, 2.6731, -0.917, -0.6753, -0.4761, -0.1091, 0.826, -0.3838, -0.1017, -0.5256, 0.3605]       |0     |
|3    |[0.

In [ ]:
n_rows = train_class.count()
train_data = train_class.select("index", "features")

In [ ]:
print(f"Number of rows in df: {n_rows}")

Number of rows in df: 8949


**<h1> <font color="#FF416C">K-MEANS CLUSTERING**

In [ ]:
def euclidean_distance(arr1, arr2):
    # Convert input arrays to dense vectors
    vec1 = Vectors.dense(arr1)
    vec2 = Vectors.dense(arr2)

    # Calculate the squared Euclidean distance between vectors
    return float(vec1.squared_distance(vec2))

# Register the function as a UDF
euclidean_distance_udf = udf(euclidean_distance, DoubleType())

In [ ]:
def assign_cluster_df(data, centroids):
    # Calculate distances for each centroid
    for i, centroid in enumerate(centroids):
        centroid_features = centroid.select("features").first()["features"]
        distance_column = f"distance_{i}"

        data = data.withColumn(
            distance_column,
            euclidean_distance_udf("features", lit(centroid_features))
        )

    # Create a list of distance columns
    distance_columns = [col(f"distance_{i}") for i in range(len(centroids))]



    # Find the minimum distance among all distance columns
    min_distance_col = least(*distance_columns)

    # Add the 'cluster' column based on the minimum distance
    data = data.withColumn("cluster", min_distance_col)

    # Create a list to hold the conditions for each cluster
    conditions = [col(f"distance_{i}") == min_distance_col for i in range(K)]

    # Use a loop to dynamically create when conditions
    cluster_expr = None
    for i in range(K):
        cluster_expr = when(conditions[i], i) if cluster_expr is None else cluster_expr.when(conditions[i], i)

    # Add the new "cluster" column
    data = data.withColumn("cluster", cluster_expr)

    # Construct the list of columns to select
    selected_columns = ["index", "features", "cluster"] + [f"distance_{i}" for i in range(K)]

    # Select the desired columns
    selected_data = data.select(selected_columns)

    return selected_data

In [ ]:
def update_centroids(data, centroids):
    centroids_new = [
        data.filter(data.cluster == i)
            .selectExpr(*[f"avg(features[{j}]) as mean_feature_{j + 1}" for j in range(len(data.select("features").first()[0]))])
            .select(array(*[col(f"mean_feature_{j + 1}") for j in range(len(data.select("features").first()[0]))]).alias("features"))
            .first().features
        for i in range(K)
    ]

    for i in range(K):
        centroids[i] = centroids[i].withColumn("features", lit(centroids_new[i]))

    return centroids

In [ ]:
def is_converged(prev_centroids, centroids, threshold=0.01):
    # false if lengths are not equal
    if len(prev_centroids) != len(centroids):
        return False

    # iterate over each entry of clusters and check if the distance is below the threshold
    for c in range(len(centroids)):
        centroid_features = centroids[c].select("features").first()["features"]
        prev_centroid_features = prev_centroids[c].select("features").first()["features"]
        distance = euclidean_distance(centroid_features, prev_centroid_features)
        if distance > threshold:
            return False

    return True

In [ ]:
def k_means(data, k=3, max_iterations=50, threshold=0.01):

    centroids = []
    # initialization, assign random data points as centroids
    count = 0
    hash_map = dict()
    while count < k:
        random_data_idx = np.random.randint(0, n_rows)

        if random_data_idx not in hash_map:
            count += 1
            hash_map[random_data_idx] = True
            centroids.append(train_data.filter(train_data.index == random_data_idx))

    iter_count = 0
    prev_centroids = []

    # run the iterations until not converged or until the max iteration is not reached
    while (is_converged(prev_centroids, centroids, threshold)) == False and (iter_count < max_iterations):

        print("running iteration " + str(iter_count))

        clusters = assign_cluster_df(train_data, centroids)

        # Create a copy of centroids to store as prev_centroids
        prev_centroids = [centroids[i].select("features") for i in range(k)]

        centroids = update_centroids(clusters, centroids)

        iter_count = iter_count + 1

    if (iter_count == max_iterations):
        print("max iterations reached, K means not converged")
    else:
        print("converged")

    return clusters

In [ ]:
# Run k-means with the customer data
clusters= k_means(train_class, k=K, threshold=_THRESHOLD)

running iteration 0


running iteration 1
running iteration 2
running iteration 3


running iteration 4
running iteration 5
running iteration 6
running iteration 7
running iteration 8
running iteration 9
running iteration 10
running iteration 11
running iteration 12


running iteration 13
running iteration 14
running iteration 15
running iteration 16
running iteration 17
running iteration 18
converged


In [ ]:
# Print the results
for c in range(K):
    print(f"Cluster {c + 1}: {clusters.select('cluster').filter(clusters.cluster == c).count()} customers")

Cluster 1: 1535 customers
Cluster 2: 5889 customers
Cluster 3: 1525 customers


# Evaluation Metrics

In [ ]:
# Create a VectorAssembler to assemble selected columns into a dense vector
vec_assembler = VectorAssembler(inputCols=selected_columns, outputCol="features1")

# Apply VectorAssembler to the DataFrame
clusters1 = vec_assembler.transform(train)

clusters1 = clusters1.select("index", "features1")

In [ ]:
# Perform the join and add a new column to df1
result = clusters.join(clusters1, clusters["index"] == clusters1["index"], "inner").drop(clusters1["index"])

In [ ]:
evaluator = ClusteringEvaluator(featuresCol = "features1")

evaluator.setPredictionCol("cluster")

evaluator.evaluate(result)

23/12/20 22:47:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


0.379926851555233

## Compare to Library from Spark

In [ ]:
clusters2 = clusters1.select(col("features1").alias("features"))

kmeans = KMeans(k=3)

kmeans.setSeed(1)

model = kmeans.fit(clusters2)

In [ ]:
pred = model.transform(clusters2)

In [ ]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(pred)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()

Silhouette with squared euclidean distance = 0.26998417532412766
